In [1]:
import argparse

In [1]:
import faiss
import numpy as np
import pandas as pd
import sklearn as sk
import torch
from sklearn.cluster import Birch
from sklearn.decomposition import PCA
from collections import Counter
from sklearn.cluster import MiniBatchKMeans
import joblib

### Search test

### Load data

In [12]:
index = faiss.read_index("index.wEOS.commonvoice.lcFalse.large-v3.trained")
index.ntotal, index.d

(659389, 1280)

In [3]:
import numpy as np

# Load the .npy file
#data = np.load('Save_test_file/retrieve_start0_size659389_k5.npy')
data = np.memmap('Save_test_file/dstore_scorefilter_size0_embed1024_vals.npy', dtype=np.int32, mode='r', shape=(1, 1))
#data = np.load('Save_test_file/dstore_scorefilter_size0_embed1024_vals.npy')
# Check the type and shape
print(type(data[0][0]))
print(data.shape)
print(data)


OSError: [WinError 8] Not enough memory resources are available to process this command

In [4]:
keys = joblib.load("datastore_raw_data_commonvoice_partNone_large-v3_lcFalse.pkl")
keys

{'hidden_states': tensor([[-0.5430,  0.0615, -0.2345,  ..., -0.1176, -0.1875, -0.4331],
         [-0.5840, -0.5552, -0.1271,  ...,  0.1182, -0.0609, -0.7959],
         [-0.4229, -0.0629, -0.5059,  ...,  0.2708, -0.1753, -0.8037],
         ...,
         [-0.1028, -0.1509, -0.5933,  ..., -0.3079, -0.0583, -0.4873],
         [-0.5488,  0.3057, -0.5850,  ..., -0.1272, -0.0903, -0.8413],
         [-0.8218, -0.5474, -0.9390,  ...,  0.0231, -0.5361, -0.6562]],
        dtype=torch.float16),
 'idxs': tensor([[ 5041],
         [   11],
         [ 4956],
         ...,
         [18470],
         [   13],
         [50257]])}

In [5]:
ids = keys["idxs"]
keys = keys["hidden_states"]
keys = np.array(keys)
ids = None

In [6]:
total = index.ntotal
k = 5

In [11]:
index.d

NameError: name 'index' is not defined

In [9]:
_, i = index.search(keys, k)

In [10]:
i[:5]

array([[     0, 329529, 201661,      2, 148917],
       [     1, 142641, 329633, 315371, 110641],
       [     2, 457397, 567344, 415184, 410608],
       [     3, 534351,  30542, 359813, 101941],
       [     4, 472534, 522408,  92018, 215079]], dtype=int64)

In [11]:
joblib.dump(i, "Cluster_results/commonvoice_knn_k5.pkl")

['Cluster_results/commonvoice_knn_k5.pkl']

In [2]:
i = joblib.load("Cluster_results/commonvoice_knn_k5.pkl")

In [6]:
i

array([[     0, 329529, 201661,      2, 148917],
       [     1, 142641, 329633, 315371, 110641],
       [     2, 457397, 567344, 415184, 410608],
       ...,
       [659386, 172614, 299819, 603951, 498395],
       [659387, 613202, 612637, 594590, 651316],
       [659388, 272208,   7838, 640285, 467723]], dtype=int64)

In [10]:
i.astype(np.int64).tofile('Save_test_file/retrieve_start0_size659389_k5.npy')

In [5]:
np.save('Save_test_file/retrieve_start0_size659389_k5.npy', i)

In [ ]:
"Save_test_file/retrieve_start0_size659389_k5.npy"

In [16]:
k = 10
_, i = index.search(keys, k)

In [17]:
i[:5]

array([[     0, 329529, 201661,      2, 148917, 657279,  31684, 558672,
        535587, 291747],
       [     1, 142641, 329633, 315371, 110641, 248525, 414018, 212903,
        304386, 464806],
       [     2, 457397, 567344, 415184, 410608, 168850, 400321, 446142,
             4, 223228],
       [     3, 534351,  30542, 359813, 101941,  55067, 405899, 355297,
        121711, 483886],
       [     4, 472534, 522408,  92018, 215079, 559350, 141323,      2,
        457397, 100760]], dtype=int64)

In [18]:
joblib.dump(i, "Cluster_results/commonvoice_knn_k10.pkl")

['Cluster_results/commonvoice_knn_k10.pkl']

In [ ]:
id_data = torch.load("token_ids.wEOS.voxpopuli.lcFalse.large-v3.pt") # -> Wrong dataset!!!!!!!!!!!!!!!!!!!!
print(len(id_data))

5436281


In [3]:
id_data = torch.load("token_ids.wEOS.commonvoice.lcFalse.large-v3.pt")
print(len(id_data))

659389


In [4]:
id_data_array = np.array([item[0] for item in id_data])

In [ ]:
id_data_array[:5]

array([1711,  264, 1623,  436, 2644])

### Create workdata

In [5]:
smaller_subset = index.reconstruct_n(0, 50000)

In [5]:
subset = index.reconstruct_n(0, index.ntotal)

In [6]:
subset[:5]

array([[-0.24553172, -0.04392557, -0.42680156, ..., -0.04169421,
        -0.05567273, -0.5128853 ],
       [-0.3743793 , -0.35530722, -0.15847413, ..., -0.18148729,
        -0.14500672, -0.63844013],
       [-0.5180663 , -0.26264462, -0.48546612, ..., -0.16891247,
        -0.22027986, -0.61216444],
       [-0.67666584, -0.30429846, -0.26070958, ..., -0.5676801 ,
        -0.4938281 , -0.88617927],
       [-0.4467095 , -0.07173336, -0.16983704, ..., -0.1657219 ,
        -0.22314629, -0.07349965]], dtype=float32)

In [6]:
pca_fitted = PCA(n_components = 100).fit(subset[:50000])
pca_subset = pca_fitted.transform(subset)

In [8]:
pca_smaller_subset = pca_fitted.transform(smaller_subset)

In [7]:
pca_subset[0]

array([ 2.10043430e-01,  4.46744347e+00, -1.36255193e+00,  2.17971706e+00,
       -1.68535209e+00,  6.70866966e-02,  1.53277099e-01, -3.12414765e-01,
       -2.82439500e-01, -1.05924010e-01,  3.88055861e-01, -6.36637211e-04,
       -3.34452003e-01, -1.62829936e-01,  1.45353436e-01, -9.97934639e-02,
       -4.04473305e-01,  3.16474140e-01, -1.08694047e-01, -2.60137022e-01,
       -7.93303490e-01, -6.84321761e-01,  2.76237801e-02, -3.15200090e-02,
        1.98544323e-01, -2.16602296e-01, -3.51701975e-02,  1.18380964e-01,
        2.12264746e-01, -1.28493458e-01,  1.52172327e-01,  2.59010643e-01,
        3.99937451e-01, -9.76549089e-02,  4.96633053e-02, -8.47278982e-02,
        2.70981669e-01,  6.14365041e-02,  1.17156029e-01, -3.16132665e-01,
       -5.97410560e-01, -4.84333783e-01, -3.51927936e-01,  3.46126676e-01,
       -4.58554104e-02,  6.77273870e-02, -3.92392337e-01,  1.72150701e-01,
        2.11298317e-01,  3.41075540e-01,  8.18023980e-02,  9.85090286e-02,
        3.81193399e-01,  

In [10]:
br = Birch(threshold = 1, branching_factor = 100, n_clusters=int(len(smaller_subset) / 3))

In [ ]:
br2 = br.fit_predict(pca_smaller_subset)

c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [37]:
br3 = br2

In [ ]:
faiss.km

In [45]:
br.fit(pca_subset[:50000])

Birch(n_clusters=16666, threshold=0.8)

In [11]:
#mbk = MiniBatchKMeans(n_clusters=400000, batch_size=10000, max_iter=100)
mbk = MiniBatchKMeans(n_clusters=300000, batch_size=50, max_iter=5)
br2 = mbk.fit_predict(subset)

c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


KeyboardInterrupt: 

In [80]:
br2 = br.fit_predict(pca_subset)

MemoryError: Unable to allocate 26.1 GiB for an array with shape (3499539630,) and data type float64

In [ ]:
#prev was Birch(treshold = 2.5, branching_factor = 50, n_clusters=int(len(subset) / 3))

br = Birch(threshold = 0.5, branching_factor = 40, n_clusters=int(len(smaller_subset) / 3))
br2 = br.fit_predict(pca_smaller_subset)

In [65]:
kmeans = MiniBatchKMeans(n_clusters = int(index.ntotal / 3), batch_size=1000)
labels = kmeans.fit_predict(pca_subset)

KeyboardInterrupt: 

In [12]:
br2 = final_labels

In [81]:
subset = subset[:100000]

In [13]:
max(br2)

659385

In [14]:
len(np.unique(br2)), len(subset)

(510643, 659389)

In [15]:
def get_mode(lst):
    return Counter(lst).most_common(1)[0][0]

In [16]:
different_items = np.unique(br2)

id_list = []
vectors_list = []
for item in different_items:
    vectors_list.append(np.mean(subset[np.where(br2 == item)], axis = 0))
    id_list.append(get_mode(id_data[np.where(br2 == item)]))

In [17]:
(1 - len(different_items) / len(subset)) * 100

22.558156111187778

In [13]:
vectors_list[:5]

[array([-0.13735606, -0.13777061, -0.50512624, ..., -0.19769602,
        -0.23727849, -0.47108135], dtype=float32),
 array([ 0.11572772, -0.15828453, -0.4104382 , ..., -0.10465684,
        -0.321633  , -0.4201675 ], dtype=float32),
 array([ 0.13042326, -0.08112734, -0.3206883 , ...,  0.02642322,
        -0.28385296, -0.56407785], dtype=float32),
 array([ 0.36067563, -0.2542439 , -0.31230825, ...,  0.01055719,
        -0.35537893, -0.51643753], dtype=float32),
 array([ 0.11572772, -0.15828453, -0.4104382 , ..., -0.10724246,
        -0.3672    , -0.50291955], dtype=float32)]

In [68]:
id_list[:5]

[tensor([1009], dtype=torch.int32),
 tensor([1455], dtype=torch.int32),
 tensor([50257], dtype=torch.int32),
 tensor([854], dtype=torch.int32),
 tensor([294], dtype=torch.int32)]

In [69]:
Counter([365, 366]).most_common(1)[0][0]

365

In [70]:
id_data[np.where(br2 == 0)]

tensor([[1009]], dtype=torch.int32)

In [71]:
# for item in range(1, 3):
#     print(id_data[np.where(br2 == item)])
#     print(get_mode(id_data[np.where(br2 == item)]))

In [18]:
d = index.d
own_index = faiss.IndexFlatL2(d)
own_index.add(np.array(vectors_list))

In [32]:
O_d, O_i = own_index.search(subset[:20000], 5)
d, i = index.search(subset[:20000], 5)

In [39]:
id_data[np.where(i[0])]

tensor([[ 264],
        [1623],
        [ 436],
        [2644]], dtype=torch.int32)

In [40]:
id_data[np.where(O_i[0])]

tensor([[1711],
        [ 264],
        [1623],
        [ 436],
        [2644]], dtype=torch.int32)

In [33]:
O_i

array([[ 92508,  92575,  92480,  92974,  92407],
       [ 30677,  30556,  31064,  30593,  30934],
       [ 92850,  92974,  92419,  92407,  92575],
       ...,
       [313374, 313218, 313220, 313292, 312972],
       [243467, 243291, 243294, 243341, 243245],
       [ 50929,  50945,  50426,  50516,  50856]], dtype=int64)

In [34]:
i

array([[     0, 201661, 291747,  31684, 311439],
       [     1, 110641, 464806, 248525, 315371],
       [     2, 567344, 168850, 400321, 223228],
       ...,
       [ 19997, 153179,  65794, 290749, 356453],
       [ 19998, 330666, 591816, 282309, 613022],
       [ 19999,  73370, 536750, 125901, 333683]], dtype=int64)

In [31]:
O_i

array([[92508, 92575, 92480, 92974, 92407]], dtype=int64)

In [44]:
d

array([[ 0.      , 52.530136, 53.044857, 53.231926, 53.656265]],
      dtype=float32)

In [45]:
vector = index.reconstruct_n(0, 1)
print(vector)
O_d, O_i = own_index.search(vector, 5)
d, i = index.search(vector, 5)
i

[[-0.24553172 -0.04392557 -0.42680156 ... -0.04169421 -0.05567273
  -0.5128853 ]]


array([[     0, 201661, 291747,  31684, 311439]], dtype=int64)

In [46]:
for i in subset[:50]:
    print(len(i))

1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280


In [47]:
# total = 0
# correct = 0
# for vector in subset:
#     O_d, O_i = own_index.search(np.array([vector]), 5)
#     d, i = index.search(np.array([vector]), 5)
#     data_O = id_data[O_i[0]]
#     data_not_O = id_data[i[0]]
#     total += len(data_O)
#     correct += sum([1 for j in data_O if j in data_not_O])

In [19]:
subset_array = np.array(subset)

In [20]:
id_array = np.array(id_list)

In [21]:
og_list_array = np.array(id_data)

In [76]:
og_list_array[:5]

array([[1711],
       [ 264],
       [1623],
       [ 436],
       [2644]])

In [22]:
random_vectors = np.random.rand(20000, index.d)

In [28]:
O_i

array([[92508, 92575, 92480, 92974, 92407]], dtype=int64)

In [23]:
k = 5

O_d, O_i = own_index.search(random_vectors, k)
d, i = index.search(random_vectors, k)

In [35]:
total = 0
correct = 0

# # Preconvert to list of Counters (preserve duplicates)
# id_data_counters = [
#     Counter(item[0].tolist()) if isinstance(item, tuple) else Counter(item.tolist())
#     for item in id_data
# ]

for idx in range(20000):
    data_Own = id_array[O_i[idx]]
    data_not_Own = og_list_array[i[idx]]

    total += len(data_Own)
    
    #correct += sum([1 for item in data_not_Own if item in data_Own])

    correct += sum(1 for item in data_not_Own if any(np.array_equal(item, own_item) for own_item in data_Own))

    # for item in data_O:
    #     correct += min(data_O[item], data_not_O.get(item, 0))


In [36]:
correct / total * 100

98.708

With first 200000 subest serach and k - 10, the % correct was 44.94 %
First 10000 (k = 5) with random is 53 % correct -> 20 000: 52,95 -> if i only take k = 1 the results are higher with 73.155

In [ ]:
average_overlap(O_i, i, 5)

6.066221911496855e-06

In [ ]:
vectors_test = index.reconstruct_n(0, 3)
d_t, i_t = index.search(vectors_test, 5)
d_t_o, i_t_o = own_index.search(vectors_test, 5)

In [ ]:
i_t

array([[     0, 201661, 291747,  31684, 311439],
       [     1, 110641, 464806, 248525, 315371],
       [     2, 567344, 168850, 400321, 223228]], dtype=int64)

In [ ]:
og_list_array[i_t]

array([[[ 1711],
        [ 2296],
        [50257],
        [14001],
        [50257]],

       [[  264],
        [  281],
        [  264],
        [  264],
        [ 3315]],

       [[ 1623],
        [   11],
        [ 1062],
        [50257],
        [11277]]])

In [ ]:
id_array[i_t_o], 

(array([[[1623],
         [ 264],
         [1711],
         [ 689],
         [ 294]],
 
        [[1768],
         [3040],
         [ 294],
         [ 264],
         [  11]],
 
        [[1623],
         [ 689],
         [1711],
         [ 264],
         [2644]]]),)

In [ ]:
id_list[i_t_o[idx][0]]

tensor([1623], dtype=torch.int32)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import time

# Simulate data
X = np.random.rand(100000, 1080).astype(np.float32)

# Optional: PCA to reduce dimensions
pca = PCA(n_components=200)
X_reduced = pca.fit_transform(X)

In [90]:


# Cluster
start = time.time()
mbk = MiniBatchKMeans(n_clusters=10000, batch_size=100, max_iter=100)
labels = mbk.fit_predict(X_reduced)
end = time.time()

print(f"Time taken: {(end - start)/60:.2f} minutes")


c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


Time taken: 6.86 minutes


In [22]:
num = index.ntotal
num = 100000

In [23]:
X = np.random.randn(num, 100).astype('float32')

In [30]:
X = index.reconstruct_n(0, num)

In [31]:
X = PCA(n_components = 100).fit_transform(X)

In [23]:
# 30000 met 10000 clusters duurde 58 sec.

In [32]:
n_clusters = 30000
kmeans = faiss.Kmeans(d=100, k=n_clusters, niter=20, verbose=True, gpu=False)
kmeans.train(X)

189184.84375

In [30]:
smaller_subset = index.reconstruct_n(0, 100000)

NameError: name 'index' is not defined

In [60]:
len(np.unique(final_labels)), max(final_labels), len(subset), len(pca_subset)

(510559, 659387, 659389, 659389)

In [59]:
# # Pseudocode
# coarse_birch = Birch(threshold = 5, branching_factor = 200, n_clusters=1000).fit(pca_subset)
# labels_1 = coarse_birch.labels_

# final_labels = np.zeros_like(labels_1)
# new_label = 0
# for i in range(1000):
#     print(i)
#     sub = pca_subset[labels_1 == i]
#     if len(sub) > 1:
#         fine_kmeans = Birch(threshold = 0.5, branching_factor = 100, n_clusters=len(sub / 3 + 0.7)).fit(sub)
#         final_labels[labels_1 == i] = fine_kmeans.labels_ + new_label
#         new_label += len(sub / 3 + 0.7)
#     elif len(sub) == 1:
#         final_labels[labels_1 == i] = final_labels[labels_1 == i] + new_label
#         new_label += 1

In [9]:
import joblib

In [11]:
joblib.dump(final_labels, "labels_kmeans_smalles_dataset.pkl")

['labels_kmeans_smalles_dataset.pkl']

In [8]:
coarse_kmeans = MiniBatchKMeans(n_clusters=1000).fit(pca_subset)
labels_1 = coarse_kmeans.labels_

final_labels = np.zeros_like(labels_1)
new_label = 0
for i in range(1000):
    print(i)
    sub = pca_subset[labels_1 == i]
    if len(sub) >= 0:
        fine_kmeans = MiniBatchKMeans(n_clusters=len(sub / 3 + 0.7)).fit(sub)
        final_labels[labels_1 == i] = fine_kmeans.labels_ + new_label
        new_label += len(sub / 3 + 0.7)


c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


0


c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=2
  warnings.warn(


1
2


c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=2
  warnings.warn(
c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(


...

992
993
994


c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(


995
996
997
998


c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
c:\Users\Mikev\anaconda3\envs\faiss_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=2
  warnings.warn(


999


In [17]:
D, I = kmeans.index.search(X, 1)
I

array([[14124],
       [ 1085],
       [24423],
       ...,
       [22466],
       [18306],
       [ 6183]], dtype=int64)

## Attempt 2

In [33]:
vectors = subset[:50000]
[vectors][0][0]

array([-0.24553172, -0.04392557, -0.42680156, ..., -0.04169421,
       -0.05567273, -0.5128853 ], dtype=float32)

In [34]:
# Assume `vectors` is your numpy array of shape (600000, 1080)
n_clusters = int(len(vectors) / 5)  # You can tune this
kmeans = faiss.Kmeans(d=index.d, k=n_clusters, niter=20, verbose=True, gpu=False)
kmeans.train(vectors)

# Get cluster centroids and assign each vector to a cluster
centroids = kmeans.centroids  # Shape: (5000, 1080)
_, assignments = kmeans.index.search(vectors, 1)  # assignments.shape: (600000, 1)

In [35]:
# Replace each vector with its centroid
reduced_vectors = centroids[assignments.flatten()]  # Shape: (600000, 1080)

In [36]:
index_orig = faiss.IndexFlatL2(index.d)
index_orig.add(vectors)

D_orig, I_orig = index_orig.search(vectors[:10000], k=10)

In [37]:
index_reduced = faiss.IndexFlatL2(index.d)
index_reduced.add(reduced_vectors)

D_red, I_red = index_reduced.search(reduced_vectors[:10000], k=10)


In [38]:
def average_overlap(true_neighbors, approx_neighbors, k):
    overlaps = [
        len(set(true_neighbors[i]) & set(approx_neighbors[i])) / k
        for i in range(len(true_neighbors))
    ]
    return sum(overlaps) / len(overlaps)

# overlap_score = average_overlap(I_orig, I_red, k=10)
# print(f"Average neighbor overlap: {overlap_score:.2f}")


In [41]:
#test how usefull
len(vectors), len(reduced_vectors), index_orig.ntotal, index_reduced.ntotal

(50000, 50000, 50000, 50000)

In [ ]:
# with more clusters

In [ ]:
# Assume `vectors` is your numpy array of shape (600000, 1080)
n_clusters = int(index.ntotal / 3)  # You can tune this
kmeans = faiss.Kmeans(d=index.d, k=n_clusters, niter=20, verbose=True, gpu=False)
kmeans.train(vectors)

# Get cluster centroids and assign each vector to a cluster
centroids = kmeans.centroids  # Shape: (5000, 1080)
_, assignments = kmeans.index.search(vectors, 1)  # assignments.shape: (600000, 1)

In [ ]:
# Replace each vector with its centroid
reduced_vectors = centroids[assignments.flatten()]  # Shape: (600000, 1080)

In [ ]:
index_orig = faiss.IndexFlatL2(index.d)
index_orig.add(vectors)

D_orig, I_orig = index_orig.search(vectors[:10000], k=10)

In [ ]:
index_reduced = faiss.IndexFlatL2(index.d)
index_reduced.add(reduced_vectors)

D_red, I_red = index_reduced.search(reduced_vectors[:10000], k=10)


In [ ]:
def average_overlap(true_neighbors, approx_neighbors, k):
    overlaps = [
        len(set(true_neighbors[i]) & set(approx_neighbors[i])) / k
        for i in range(len(true_neighbors))
    ]
    return sum(overlaps) / len(overlaps)

overlap_score = average_overlap(I_orig, I_red, k=10)
print(f"Average neighbor overlap: {overlap_score:.2f}")


In [42]:
# with good intention

In [45]:
# Assume vectors is (600_000, 1080), float32
n_clusters = 10_000  # Change based on desired compression

kmeans = faiss.Kmeans(d=index.d, k=n_clusters, niter=20, verbose=True, gpu=False)
kmeans.train(vectors)

centroids = kmeans.centroids  # Shape: (n_clusters, 1080)


In [47]:
index_compressed = faiss.IndexFlatL2(index.d)
index_compressed.add(centroids)

In [ ]:
index_full = faiss.IndexFlatL2(index.d)
index_full.add(vectors)

# Use 1,000 queries to evaluate
queries = vectors[:20000]

D_orig, I_orig = index.search(queries, 10)
D_compr, I_compr = index_compressed.search(queries, 10)


In [80]:
queries = vectors[:20000]

D_orig, I_orig = index.search(queries, 10)
D_compr, I_compr = index_compressed.search(queries, 10)

In [83]:
from sklearn.metrics.pairwise import cosine_similarity

# Compare original neighbors with compressed neighbors for each query
cos_sim = [
    cosine_similarity(
        I_orig[i], I_compr[i]
        #queries[i].reshape(1, -1), 
        #centroids[I_compr[i]]
    ).mean()
    for i in range(len(queries))
]

print(f"Average cosine similarity of compressed neighbors: {np.mean(cos_sim):.4f}")


ValueError: Expected 2D array, got 1D array instead:
array=[     0. 201661. 291747.  31684. 311439. 630298. 558672. 357166. 274796.
 553438.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [53]:
index.ntotal, index_compressed.ntotal

(659389, 10000)

In [90]:
len(centroids)

10000

In [9]:
def compute_recall_at_k(original_vectors, compressed_vectors, queries, k=10):
    # Build original index
    index_orig = faiss.IndexFlatL2(original_vectors.shape[1])
    index_orig.add(original_vectors)
    D_orig, I_orig = index_orig.search(queries, k)

    # Build compressed index
    index_comp = faiss.IndexFlatL2(compressed_vectors.shape[1])
    index_comp.add(compressed_vectors)
    D_comp, I_comp = index_comp.search(queries, k)

    # Calculate recall@k
    recalls = []
    for i in range(len(queries)):
        true_neighbors = set(I_orig[i])
        retrieved_neighbors = set(I_comp[i])
        recall = len(true_neighbors & retrieved_neighbors) / k
        recalls.append(recall)

    avg_recall = np.mean(recalls)
    return avg_recall


In [9]:
original_ids = index.id_map.ids if hasattr(index, "id_map") else np.arange(index.ntotal)
original_ids

array([     0,      1,      2, ..., 659386, 659387, 659388])

In [12]:
kmeans = faiss.Kmeans(d=index.d, k=10, niter=20, verbose=True)
kmeans.train(subset[:10000])

226999.390625

In [14]:
_, cluster_assignments = kmeans.index.search(subset[:10000], 1)
compressed_vectors = kmeans.centroids

In [16]:
cluster_assignments

array([[8],
       [0],
       [1],
       ...,
       [8],
       [1],
       [6]], dtype=int64)

In [22]:
len(compressed_vectors)

10

In [10]:
from collections import defaultdict

In [11]:
num_vectors = index.ntotal
dim = index.d

# Extract vectors
vectors = subset

# If index has ID mapping, get them
original_ids = index.id_map.ids if hasattr(index, "id_map") else np.arange(num_vectors)


In [ ]:
num_clusters = 300000  # adjust as needed
kmeans = faiss.Kmeans(d=dim, k=num_clusters, niter=20, verbose=True, gpu = False)
kmeans.train(vectors)

# Assign each vector to nearest cluster
_, cluster_assignments = kmeans.index.search(vectors, 1)
cluster_assignments = cluster_assignments.flatten()


In [ ]:
cluster_to_ids = defaultdict(list)

for i, cluster_id in enumerate(cluster_assignments):
    cluster_to_ids[cluster_id].append(id_data[i])

In [ ]:
compressed_vectors = kmeans.centroids  # shape: (num_clusters, dim)
compressed_index = faiss.IndexFlatL2(dim)
compressed_index.add(compressed_vectors)


In [ ]:
# Use first ID in each cluster or majority vote or whatever strategy you prefer
cluster_rep_ids = np.array([
    cluster_to_ids[c][0] if cluster_to_ids[c] else -1
    for c in range(num_clusters)
])


In [ ]:
# Use original vectors as queries
D_orig, I_orig = index.search(vectors, 1)
D_comp, I_comp = compressed_index.search(vectors, 1)

# Get predicted IDs from compressed index (through cluster ID mapping)
predicted_ids = cluster_rep_ids[I_comp.flatten()]


In [ ]:
# Compare to true original IDs
true_ids = original_ids
accuracy = sum([predicted_ids[i] == true_ids[i] for i in range(len(true_ids))])[0] / len(true_ids)
print(f"ID match accuracy after compression: {accuracy:.8f}")

ID match accuracy after compression: 0.00000303


In [ ]:
#More neighbours
# Use original vectors as queries
D_orig, I_orig = index.search(vectors, 10)
D_comp, I_comp = compressed_index.search(vectors, 10)

# Get predicted IDs from compressed index (through cluster ID mapping)
predicted_ids = cluster_rep_ids[I_comp.flatten()]


In [ ]:
# Compare to true original IDs
true_ids = original_ids
accuracy = sum([predicted_ids[i] == true_ids[i] for i in range(len(true_ids))])[0] / len(true_ids)
print(f"ID match accuracy after compression: {accuracy:.8f}")

In [66]:
index_copy = index

In [12]:
type(subset)

numpy.ndarray

In [79]:
from joblib import Memory
import joblib

In [78]:
memory = Memory(location="cache_dir", verbose=0)

In [80]:
joblib.dump(all_25_neighbours, "my_cache.pkl")

['my_cache.pkl']

In [82]:
loaded_var = joblib.load("my_cache.pkl")
loaded_var[:5]

array([[     0, 201661, 291747,  31684, 311439, 630298, 558672, 357166,
        274796, 553438, 329529, 368141, 625081, 176188, 106271, 657279,
        526044, 299422, 569377, 604911, 148917, 568547, 630194, 591351,
        594239],
       [     1, 110641, 464806, 248525, 315371, 180351, 304386, 565979,
        571782, 333975, 248851, 586170, 142641, 359209, 433525, 273484,
        240871, 220752, 507213, 620899, 120993, 342248,  42545, 288625,
        543440],
       [     2, 567344, 168850, 400321, 223228, 446142, 260993, 410608,
        623779, 415184, 448322, 273077, 302907, 208914, 174758, 229648,
        567562, 298848, 149657, 373664, 280845, 199589, 114283, 256087,
        238791],
       [     3,  30542, 412295, 477278,  50110, 124166, 498642, 314475,
        193903, 326491, 647359, 542087, 235290, 548497, 405899, 569755,
        541702, 226915, 508137, 470671, 289586, 335463, 162492,  74894,
        193640],
       [     4, 119630, 141323, 522408, 304308, 559350, 490537, 4359

In [9]:
bullshit, all_25_neighbours = index.search(np.array(subset), 25)
all_25_neighbours[:5]

array([[     0, 201661, 291747,  31684, 311439, 630298, 558672, 357166,
        274796, 553438, 329529, 368141, 625081, 176188, 106271, 657279,
        526044, 299422, 569377, 604911, 148917, 568547, 630194, 591351,
        594239],
       [     1, 110641, 464806, 248525, 315371, 180351, 304386, 565979,
        571782, 333975, 248851, 586170, 142641, 359209, 433525, 273484,
        240871, 220752, 507213, 620899, 120993, 342248,  42545, 288625,
        543440],
       [     2, 567344, 168850, 400321, 223228, 446142, 260993, 410608,
        623779, 415184, 448322, 273077, 302907, 208914, 174758, 229648,
        567562, 298848, 149657, 373664, 280845, 199589, 114283, 256087,
        238791],
       [     3,  30542, 412295, 477278,  50110, 124166, 498642, 314475,
        193903, 326491, 647359, 542087, 235290, 548497, 405899, 569755,
        541702, 226915, 508137, 470671, 289586, 335463, 162492,  74894,
        193640],
       [     4, 119630, 141323, 522408, 304308, 559350, 490537, 4359

In [60]:
index_neighbours_to_25 = [id_data_array[inner_list][:2] for inner_list in all_25_neighbours]
index_neighbours_to_25[0]

array([1711, 2296])

In [61]:
identical_rows = [i for i in range(len(index_neighbours_to_25)) if len(np.unique(index_neighbours_to_25[i])) == 1]
non_identical_rows = [i for i in range(len(index_neighbours_to_25)) if len(np.unique(index_neighbours_to_25[i])) != 1]
non_identical_rows_and_rest = non_identical_rows + [i for i in range(len(index_neighbours_to_25), index.ntotal)]

In [62]:
identical_rows[:5], non_identical_rows[:5], len(non_identical_rows), len(identical_rows), len(non_identical_rows_and_rest), len(subset), index.ntotal

([15, 59, 154, 736, 742],
 [0, 1, 2, 3, 4],
 651915,
 7474,
 651915,
 659389,
 659389)

In [63]:
non_id_keys = subset[non_identical_rows]
len(non_id_keys)

651915

In [67]:
index_copy.remove_ids(faiss.IDSelectorBatch(np.array(identical_rows)))

7474

In [70]:
index_copy.ntotal, index.ntotal

(651915, 659389)

In [71]:
_, true_ids_i = index.search(subset[:50000], 10)

In [72]:
_, predicted_ids_i = index_copy.search(subset[:50000], 10)

In [73]:
true_ids = [id_data_array[i] for i in true_ids_i]

In [74]:
id_data_new_index = id_data_array[non_identical_rows_and_rest]
len(id_data_new_index)

651915

In [75]:
predicted_ids = [id_data_array[i] for i in predicted_ids_i] #miss moet id_data_array -> id_data_new_index worden

In [76]:
accuracy = sum([predicted_ids[i] == true_ids[i] for i in range(len(true_ids))])[0] / len(true_ids) * 100
accuracy

99.768